In [79]:
import pandas as pd
import numpy as np
import warnings
import scipy as sp
from datetime import datetime
from IPython.display import display
import plotly
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
from IPython.core.display import display, HTML
from plotly.subplots import make_subplots
from IPython.display import display
autoscroll_set_on_load = True
autoscroll_starting_threshold = -1
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df = pd.read_excel('data/LIGHT4DEAF_Neurocognition_Avril_2020_1_FULL_ADULTES_20200504.xlsx')

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [80]:
df = df[df["Investigateur"].notnull()]

cardinal = len(df)

now = datetime.now()
df['Date de naissance du patient'] = pd.to_datetime(df['Date de naissance du patient'], format='%m/%d/%y')
df['Date de naissance du patient'] = df['Date de naissance du patient'].where(df['Date de naissance du patient'] < now, df['Date de naissance du patient'] -  np.timedelta64(100, 'Y')) 
df['age'] = (now - df['Date de naissance du patient']).astype('<m8[Y]')

#Patients adultes : type
#df_type1 = df[df["Type"]=='Type 1']
#df_type2 = df[df["Type"]=='Type 2']
#df_type3 = df[df["Type"]=='Type 3']
#df_type_nan = df[df["Type"].isna()]

In [81]:
df = df[df["Type"]=='Type 2']

In [82]:
display(HTML('<h1>L i g h t 4 d e a f</h1>'))
display(HTML('<h2>Neurocognition - WAIS</h2>'))
display(HTML("<h3>Description de l'échantillon</h3>"))
display(HTML("<h4>1. Données : </h4>"))


In [83]:
cols = df.columns.values.tolist()
cols.index('WAIS')
cols.index('Autre commentaire WAIS')
arr = cols[cols.index('WAIS'):cols.index('Autre commentaire WAIS')]
arr.insert(0,"ID_SECONDAIRE")
arr.insert(1,"age")
arr.insert(2,"Type")

In [84]:
df_wais = df.loc[:,arr]

In [85]:
df_wais.loc[df_wais['Type']=="Type 2", 'Type_color']="#B3D9FF"
#df_wais.loc[df_wais['Type']=="Type 1", 'Type_color']="#33ffc9"
#df_wais.loc[df_wais['Type']=="Type 3", 'Type_color']="#FFC433"
#df_wais.loc[df_wais['Type'].isna(), 'Type_color']="#C5C5C5"
#df_wais['Type'].fillna('Type inconnu', inplace= True)

In [86]:
display(HTML('<p>Nombre de patients Type 2 ayant effectué le test (WAIS = Oui) : '+str(len(df_wais[df_wais['WAIS']=='Oui']))+'/'+str(cardinal)+' patients</p>'))

In [87]:
columns= ['Calcul','Note standard Similitudes','Note standard Vocabulaire', 'Note standard Information',
                             'Note standard Cubes','Note standard Matrices', 'Note standard Puzzles visuels',
                              'Note standard Mémoires des chiffres', 'Note standard Arithmétique',
                                'Note standard Symboles','Note standard Code'
                             ]
myarr = [['Effectif', 'Moyenne', 'Moy. âge', 'Médiane', 'Min.', 'Max.']]
for i in columns[1:]:
    effectif = len(df_wais[df_wais[i].notna()])
    avg_age = round(df_wais[df_wais[i].notna()]['age'].mean())
    avg = round(df_wais[i].mean(),2)
    median = round(df_wais[i].median(),2)
    max_val = df_wais[i].max()
    min_val = df_wais[i].min()
    myarr.append([effectif, avg, avg_age, median, max_val, min_val])

wais_test =  go.Table(
                header=dict(values=list(columns),
                align='left'),
                cells=dict(values= myarr,align='left'))

fig = go.Figure(data=wais_test)
fig.update_layout(width=1000, height=400)
fig.show()

In [88]:
display(HTML('<h4>2. Tests </h4>'))
display(HTML('<h5>2.1. WAIS IVC</h5>'))

In [89]:
age_moy = df_wais[df_wais['Total des notes standards ICV'].notna()]['age'].mean()

In [90]:
display(HTML("<p>Moyenne d'âge :"+str(round(age_moy,2))+"</p>"))

In [91]:
df_wais_sub= df_wais.iloc[:,[0,1,2,-1,6,7,8,9,10,11,12,13]]

In [92]:
subplot_titles = df_wais_sub.columns.values.tolist()

In [93]:
fig_wais_sub = make_subplots(
    rows=4, cols=2,
    shared_xaxes=False,
    row_heights=[0.20, 0.20, 0.20, 0.20],
    column_widths=[0.5, 0.5],
    subplot_titles= subplot_titles[4:],
    x_title='Âge',
    y_title='Score',
    vertical_spacing= 0.04 )
df_wais_sub = df_wais_sub.sort_values(by=['age'])

In [94]:
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[4]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=1, col=1)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[5]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
              row=1, col=2)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[6]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
              row=2, col=1)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[7]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=2, col=2)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[8]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=3, col=1)
                                                             
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[9]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=3, col=2)
    
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[10]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=4, col=1)
    
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[11]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=4, col=2)
    
fig_wais_sub.update_layout(height=1500, width=800,
                   showlegend=False
            )

In [95]:
df_wais_sub_agg = df_wais_sub.agg(['min', 'max', 'mean', 'median', 'count'])
#
df_wais_sub_agg.index.name = "Aggrégation"
df_wais_sub_agg.reset_index(inplace=True)
#
df_wais_sub_agg = df_wais_sub_agg.replace({'Aggrégation' : {"min":"Min","max":"Max","mean":"Moyenne", "median":"Médiane", "count":"Effectif"}})

In [96]:
df_wais_sub_agg = df_wais_sub_agg.drop(columns= ['ID_SECONDAIRE','Type', 'Type_color', 'age' ])

In [97]:
df_wais_sub_agg = df_wais_sub_agg.round(2)

In [98]:
wais_sub =  go.Table(
                header=dict(values=list(df_wais_sub_agg.columns[:]),
                align='left'),
                cells=dict(values= [df_wais_sub_agg.iloc[:,0], df_wais_sub_agg.iloc[:,1], df_wais_sub_agg.iloc[:,2], df_wais_sub_agg.iloc[:,3]
                           , df_wais_sub_agg.iloc[:,4], df_wais_sub_agg.iloc[:,5], df_wais_sub_agg.iloc[:,6], df_wais_sub_agg.iloc[:,7]
                           , df_wais_sub_agg.iloc[:,8]], align='left'))

In [99]:
fig = go.Figure(data=wais_sub)
fig.update_layout(width=1000, height=1000)
fig.show()

In [100]:
display(HTML('<h5>2.1. WAIS IRP</h5>'))

In [101]:
age_moy = df_wais[df_wais['Total des notes standards IRP'].notna()]['age'].mean()

In [102]:
display(HTML("<p>Moyenne d'âge : "+str(round(age_moy,2))+"</p>"))

In [103]:
df_wais_sub= df_wais.iloc[:,[0,1,2,-1,14, 15, 16, 17, 18, 19, 20, 21]]

In [104]:
subplot_titles = df_wais_sub.columns.values.tolist()

In [105]:
fig_wais_sub = make_subplots(
    rows=4, cols=2,
    shared_xaxes=False,
    row_heights=[0.20, 0.20, 0.20, 0.20],
    column_widths=[0.5, 0.5],
    subplot_titles= subplot_titles[4:],
    x_title='Âge',
    y_title='Score',
    vertical_spacing= 0.04 )
df_wais_sub = df_wais_sub.sort_values(by=['age'])

In [106]:
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[4]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=1, col=1)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[5]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
              row=1, col=2)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[6]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
              row=2, col=1)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[7]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=2, col=2)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[8]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=3, col=1)
                                                             
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[9]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=3, col=2)
    
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[10]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=4, col=1)
    
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[11]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=4, col=2)
    
fig_wais_sub.update_layout(height=1500, width=800,
                   showlegend=False
            )

In [107]:
df_wais_sub_agg = df_wais_sub.agg(['min', 'max', 'mean', 'median', 'count'])
#
df_wais_sub_agg.index.name = "Aggrégation"
df_wais_sub_agg.reset_index(inplace=True)
#
df_wais_sub_agg = df_wais_sub_agg.replace({'Aggrégation' : {"min":"Min","max":"Max","mean":"Moyenne", "median":"Médiane", "count":"Effectif"}})

In [108]:
df_wais_sub_agg = df_wais_sub_agg.drop(columns= ['ID_SECONDAIRE','Type', 'Type_color', 'age' ])

In [109]:
df_wais_sub_agg = df_wais_sub_agg.round(2)

In [110]:
wais_sub =  go.Table(
                header=dict(values=list(df_wais_sub_agg.columns[:]),
                align='left'),
                cells=dict(values= [df_wais_sub_agg.iloc[:,0], df_wais_sub_agg.iloc[:,1], df_wais_sub_agg.iloc[:,2], df_wais_sub_agg.iloc[:,3]
                           , df_wais_sub_agg.iloc[:,4], df_wais_sub_agg.iloc[:,5], df_wais_sub_agg.iloc[:,6], df_wais_sub_agg.iloc[:,7]
                           , df_wais_sub_agg.iloc[:,8]], align='left'))

In [111]:
fig = go.Figure(data=wais_sub)
fig.update_layout(width=1000, height=400)
fig.show()

In [112]:
display(HTML('<h5>2.1. WAIS IMT</h5>'))

In [113]:
age_moy = df_wais[df_wais['Total des notes standards IMT'].notna()]['age'].mean()

In [114]:
display(HTML("<p>Moyenne d'âge : "+str(round(age_moy,2))+"</p>"))

In [115]:
df_wais_sub= df_wais.iloc[:,[0,1,2,-1,22, 23, 24, 25, 26, 27, 28]]

In [116]:
subplot_titles = df_wais_sub.columns.values.tolist()

In [117]:
fig_wais_sub = make_subplots(
    rows=4, cols=2,
    shared_xaxes=False,
    row_heights=[0.20, 0.20, 0.20, 0.20],
    column_widths=[0.5, 0.5],
    subplot_titles= subplot_titles[4:],
    x_title='Âge',
    y_title='Score',
    vertical_spacing= 0.04 )
df_wais_sub = df_wais_sub.sort_values(by=['age'])

In [118]:
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[4]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=1, col=1)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[5]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
              row=1, col=2)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[6]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
              row=2, col=1)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[7]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=2, col=2)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[8]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=3, col=1)
                                                             
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[9]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=3, col=2)
    
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[10]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=4, col=1)
    
fig_wais_sub.update_layout(height=1500, width=800,
                   showlegend=False
            )

In [119]:
df_wais_sub_agg = df_wais_sub.agg(['min', 'max', 'mean', 'median', 'count'])
#
df_wais_sub_agg.index.name = "Aggrégation"
df_wais_sub_agg.reset_index(inplace=True)
#
df_wais_sub_agg = df_wais_sub_agg.replace({'Aggrégation' : {"min":"Min","max":"Max","mean":"Moyenne", "median":"Médiane", "count":"Effectif"}})

In [120]:
df_wais_sub_agg = df_wais_sub_agg.drop(columns= ['ID_SECONDAIRE','Type', 'Type_color', 'age' ])

In [121]:
df_wais_sub_agg = df_wais_sub_agg.round(2)

In [122]:
wais_sub =  go.Table(
                header=dict(values=list(df_wais_sub_agg.columns[:]),
                align='left'),
                cells=dict(values= [df_wais_sub_agg.iloc[:,0], df_wais_sub_agg.iloc[:,1], df_wais_sub_agg.iloc[:,2], df_wais_sub_agg.iloc[:,3]
                           , df_wais_sub_agg.iloc[:,4], df_wais_sub_agg.iloc[:,5], df_wais_sub_agg.iloc[:,6], df_wais_sub_agg.iloc[:,7]], align='left'))

In [123]:
fig = go.Figure(data=wais_sub)
fig.update_layout(width=1000, height=400)
fig.show()

In [124]:
display(HTML('<h5>2.1. WAIS IVT</h5>'))

In [125]:
age_moy = df_wais[df_wais['Total des notes standards IVT'].notna()]['age'].mean()

In [126]:
display(HTML("<p>Moyenne d'âge : "+str(round(age_moy,2))+"</p>"))

In [127]:
df_wais_sub= df_wais.iloc[:,[0,1,2,-1, 29, 30, 31, 32, 33, 34, 35]]

In [128]:
subplot_titles = df_wais_sub.columns.values.tolist()

In [129]:
fig_wais_sub = make_subplots(
    rows=4, cols=2,
    shared_xaxes=False,
    row_heights=[0.20, 0.20, 0.20, 0.20],
    column_widths=[0.5, 0.5],
    subplot_titles= subplot_titles[4:],
    x_title='Âge',
    y_title='Score',
    vertical_spacing= 0.04 )
df_wais_sub = df_wais_sub.sort_values(by=['age'])

In [130]:
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[4]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=1, col=1)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[5]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
              row=1, col=2)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[6]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
              row=2, col=1)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[7]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=2, col=2)

fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[8]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=3, col=1)
                                                             
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[9]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=3, col=2)
    
fig_wais_sub.add_trace(go.Scatter(x= df_wais_sub['age'], y=df_wais_sub[subplot_titles[10]], mode="markers+text",
            marker={'size': 10, "color": df_wais_sub['Type_color'] },
            hovertext= df_wais_sub['ID_SECONDAIRE'] +", "+ df_wais_sub['Type']),
            row=4, col=1)
    
fig_wais_sub.update_layout(height=1500, width=800,
                   showlegend=False
            )

In [131]:
df_wais_sub_agg = df_wais_sub.agg(['min', 'max', 'mean', 'median', 'count'])
#
df_wais_sub_agg.index.name = "Aggrégation"
df_wais_sub_agg.reset_index(inplace=True)
#
df_wais_sub_agg = df_wais_sub_agg.replace({'Aggrégation' : {"min":"Min","max":"Max","mean":"Moyenne", "median":"Médiane",  "count":"Effectif"}})

In [132]:
df_wais_sub_agg = df_wais_sub_agg.drop(columns= ['ID_SECONDAIRE','Type', 'Type_color', 'age' ])

In [133]:
df_wais_sub_agg = df_wais_sub_agg.round(2)

In [134]:
wais_sub =  go.Table(
                header=dict(values=list(df_wais_sub_agg.columns[:]),
                align='left'),
                cells=dict(values= [df_wais_sub_agg.iloc[:,0], df_wais_sub_agg.iloc[:,1], df_wais_sub_agg.iloc[:,2], df_wais_sub_agg.iloc[:,3]
                           , df_wais_sub_agg.iloc[:,4], df_wais_sub_agg.iloc[:,5], df_wais_sub_agg.iloc[:,6], df_wais_sub_agg.iloc[:,7]], align='left'))

In [135]:
fig = go.Figure(data=wais_sub)
fig.update_layout(width=1000, height=1000)
fig.show()

In [136]:
display(HTML('<h4>4. WAIS complet</h4>'))

In [137]:
wais_complet = len(df_wais[(df_wais['Total des notes standards ICV'].notna())&
                       (df_wais['Total des notes standards IVT'].notna())&
                        (df_wais['Total des notes standards IMT'].notna())&
                        (df_wais['Total des notes standards IRP'].notna())])

ICV_IVT = len(df_wais[(df_wais['Total des notes standards ICV'].notna())&
                       (df_wais['Total des notes standards IVT'].notna())])

ICV_IMT = len(df_wais[(df_wais['Total des notes standards ICV'].notna())&
                       (df_wais['Total des notes standards IMT'].notna())])

ICV_IRP = len(df_wais[(df_wais['Total des notes standards ICV'].notna())&
                       (df_wais['Total des notes standards IRP'].notna())])

IVT_IMT = len(df_wais[(df_wais['Total des notes standards IVT'].notna())&
                       (df_wais['Total des notes standards IMT'].notna())])

IVT_IRP = len(df_wais[(df_wais['Total des notes standards IVT'].notna())&
                       (df_wais['Total des notes standards IRP'].notna())])

IMT_IRP = len(df_wais[(df_wais['Total des notes standards IMT'].notna())&
                       (df_wais['Total des notes standards IRP'].notna())])

ICV_IVT_IMT = len(df_wais[(df_wais['Total des notes standards ICV'].notna())&
                       (df_wais['Total des notes standards IVT'].notna())&
                        (df_wais['Total des notes standards IMT'].notna())])

ICV_IVT_IRP = len(df_wais[(df_wais['Total des notes standards ICV'].notna())&
                       (df_wais['Total des notes standards IVT'].notna())&
                        (df_wais['Total des notes standards IRP'].notna())])

ICV_IMT_IRP = len(df_wais[(df_wais['Total des notes standards ICV'].notna())&
                       (df_wais['Total des notes standards IMT'].notna())&
                        (df_wais['Total des notes standards IRP'].notna())])



In [138]:
wais_sub =  go.Table(
                header=dict(values=list(['Tests effectués', 'Effectif']),
                align='left'),
                cells=dict(values= [['ICV+IVT', 'ICV+IMT', 'ICV+IRP', 'IVT+IMT', 'IVT+IRP', 'IMT+IRP',
                                     'ICV+IVT+IMT', 'ICV+IVT+IRP', 'ICV+IMT+IRP',
                                     'ICV+IVT+IMT+IRP'],
                                    [ICV_IVT, ICV_IMT, ICV_IRP, IVT_IMT, IVT_IRP, IMT_IRP, ICV_IVT_IMT, ICV_IVT_IRP, ICV_IMT_IRP, wais_complet]], align='left'))

In [139]:
fig = go.Figure(data=wais_sub)
fig.show()